# Анализ результатов A/B теста

In [99]:
# Подключаем необходимые библиотеки для работы с данными

import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom
from statsmodels.stats.proportion import proportions_ztest

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [3]:
# Загружаем данные из csv для работы

groups = pd.read_csv('groups.csv', sep=';')
active_studs = pd.read_csv('active_studs.csv', sep=';')
checks = pd.read_csv('checks.csv', sep=';')

In [45]:
# Для анализа данных необходимо понять к какой из тестовых групп относятся активные пользователи, 
# для этого объединяем данные по активным пользователям с данным о распределении по группам

data = active_studs.merge(groups, how='left', left_on='student_id', right_on='id')

In [46]:
# Объединяем предыдущий датафрейм с информацией по оплатам и смотрим на получившуюся таблицу 

data = result.merge(checks, how='left', left_on='student_id', right_on='student_id')
data

,student_id,id,grp,rev_x,rev_y
0,581585,581585,A,NaN,NaN
1,5723133,5723133,A,NaN,NaN
2,3276743,3276743,B,NaN,NaN
3,4238589,4238589,A,NaN,NaN
4,4475369,4475369,B,NaN,NaN
...,...,...,...,...,...
8336,1794344,1794344,B,NaN,NaN
8337,296883,296883,B,NaN,NaN
8338,3598448,3598448,B,NaN,NaN
8339,3610547,3610547,B,290.0,290.0


In [51]:
# Смотрим как рапределились активные пользователи по группам

users = data.groupby('grp')['student_id'].count()

active_user_a = users[0]
active_user_b = users[1]

users

grp
A    1538
B    6803
Name: student_id, dtype: int64

In [52]:
# Смотрим как рапределилось количество платежей по группам активных пользователей

success = res2.groupby('grp')['rev'].count()

success_user_a = success[0]
success_user_b = success[1]

success

grp
A     78
B    314
Name: rev, dtype: int64

In [58]:
# Сформируем сводную таблицу с ключевыми значениями

ab_summary = res2.pivot_table(values='id', index='grp', aggfunc='count')
ab_summary['success'] = res2.pivot_table(values='rev', index='grp', aggfunc='count')
ab_summary = ab_summary.rename(columns={'id': 'total'})
ab_summary['cr'] = ab_summary['success'] / ab_summary['total']

ab_summary

,total,success,cr
grp,,,
A,1538,78,0.050715
B,6803,314,0.046156


### Резюме

На основании полученных предварительных данных можно сделать вывод, что анализировать результаты A/B теста будет некорректно, так как очевидно нарушено одно из главных требований к тесту – равномерность распределения пользователей на тестовую и контрольную выборку.

В контрольной группе у нас оказалось 1538 пользователей, в то время как в тестовой почти в 4,5 раза больше – 6803.

Можно сделать предположение, что распределение пользователей на контрольную и тестовую выборки было проведено заранее, до начала теста, что подтверждается исходными данными (есть большая выборка пользователей распределенных по группам, но далеко не все из них приняли участие в тесте). При проведении A/B теста необходимо случайным образом распределять пользователей в одну из выборок непосредственно перед тем как показать пользователю исследуемый или контрольный вариант платежной страницы.


Исходя из того, что нарушено главное требование к распределению пользователей на контрольную и тестовую выборки считать результат теста достоверным нельзя, как и ориентироваться на него при принятии решений.